In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter

import functools

import math

import matplotlib.pyplot as plt

In [6]:
inputFile = "water-treatmennt-original-marked.csv"

dataFrame = pd.read_csv(inputFile, header = 0, sep = ';')
print(dataFrame.shape)
data = dataFrame.values
# print(data)

numSamples = dataFrame.shape[0]
print("Num samples == " + str(numSamples))
numFeatures = dataFrame.shape[1]  #Last feature used as class marker, first feature is sampling date
print("Num features == " + str(numFeatures))

(528, 40)
Num samples == 528
Num features == 40
